# 尝试写一个整形权重CNN的预测

In [1]:
%pdb 1
from __future__ import print_function
import numpy as np
import struct

import time
import pickle
import os

Automatic pdb calling has been turned ON


In [2]:
np.random.seed(1337)  # for reproducibility
nb_classes = 34
nb_epoch = 30
batch_size = 128

image_higth, image_width = 15, 15
# lr = [0.05, 0.01, 0.005]
layer1 = 10
hidden1 = 40
region = 3

# 模型权重根目录
model_root_path = '/home/jdwang/PycharmProjects/digitRecognition/int_weight_predict/modelAndData1122/'
model_file_list_path = os.listdir(os.path.join(model_root_path, 'model'))


定义必要的函数

In [3]:
%%time
def load_valdata(version='1122'):
    """读取不同版本的测试集
    1120 - 对应 /home/jdwang/PycharmProjects/digitRecognition/int_weight_predict/modelAndData1120/Data/
    1122 - 对应 /home/jdwang/PycharmProjects/digitRecognition/int_weight_predict/modelAndData1122/Data/

    :param dir_path: str
    :param version: str
    :return:
    """

    # 数据集根目录
    data_root_path = '/home/jdwang/PycharmProjects/digitRecognition/int_weight_predict/'

    # 读取验证数据、测试数据
    if version == '1122':
        val_file_path = os.path.join(data_root_path, 'modelAndData1122/Data/', 'TrainSet_trainAndVal_testSet.pickle')
        other_file_path = os.path.join(data_root_path, 'modelAndData1122/Data/', 'Olddata_TestSet.pickle')
        with open(val_file_path, 'rb') as train_file:
            train_X = pickle.load(train_file)
            train_y = pickle.load(train_file)
            val_X, val_y = None, None
            test_X = pickle.load(train_file)
            test_y = pickle.load(train_file)
    elif version == '1120':
        val_file_path = os.path.join(data_root_path, 'modelAndData1120/Data/', 'valSet&TestSet.pickle')
        other_file_path = os.path.join(data_root_path, 'modelAndData1120/Data/', 'Olddata_TestSet.pickle')
        with open(val_file_path, 'rb') as train_file:
            val_X = pickle.load(train_file)
            val_y = pickle.load(train_file)
            test_X = pickle.load(train_file)
            test_y = pickle.load(train_file)
    else:
        raise NotImplementedError

    with open(other_file_path, 'rb') as otherFile:
        other_X = pickle.load(otherFile)
        other_y = pickle.load(otherFile)

    return (val_X, val_y), (test_X, test_y), (other_X, other_y)


def Net_model(layer1, hidden1, region, rows, cols, nb_classes, lr=0.01, decay=1e-6, momentum=0.9):
    from keras.models import Sequential
    from keras.layers.core import Dense, Dropout, Activation, Flatten
    from keras.layers.convolutional import Convolution2D, MaxPooling2D
    from keras.optimizers import SGD
    from keras import backend as K
    model = Sequential()

    model.add(Convolution2D(layer1, region, region,
                            border_mode='valid',
                            input_shape=(1, rows, cols)))

    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())  # 平铺

    model.add(Dense(hidden1))  # Full connection 1:  1000
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))

    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    # 获取CNN的中间结果
    mid_output = K.function(inputs=[
        model.layers[0].input,
        K.learning_phase(),
    ],
        outputs=[
            model.layers[-9].output,
            model.layers[-8].output,
            model.layers[-7].output,
            model.layers[-6].output,
            model.layers[-5].output,
            model.layers[-4].output,
            model.layers[-3].output,
            model.layers[-2].output,
            model.layers[-1].output,
        ]
    )

    sgd = SGD(lr=lr, decay=decay, momentum=momentum, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=["accuracy"])

    return model, mid_output


def test_model(model_file, X_test, Y_test):
    # 加载模型架构
    # 这里的 lr 设置什么不影响
    model, mid_output = Net_model(layer1, hidden1, region, image_higth, image_width, nb_classes=34, lr=0)
    model.load_weights(model_file)

    # 预测
    predicted = model.predict_classes(X_test, verbose=0)

    return count_result(predicted, Y_test)


def count_result(predicted, Y_test):
    '''统计预测情况，包括混淆集个数等

    :return:
    '''
    # 统计混淆集
    badcase = {}
    for i in range(0, len(Y_test)):
        if (tramsform(predicted[i]) in ['1', 'I'] and tramsform(Y_test[i]) in ['1', 'I']):
            # 1跟I区分，只要 是 测试 成1或I，而实际值是 1或I都算对
            predicted[i] = Y_test[i]
        if predicted[i] != Y_test[i]:
            ch1 = tramsform(Y_test[i])
            ch2 = tramsform(predicted[i])
            string = ','.join(sorted([ch1, ch2]))

            if badcase.has_key(string):
                badcase[string] += 1
            else:
                badcase[string] = 1

    # 计算测试准确率
    test_accuracy = np.mean(np.equal(predicted, Y_test))
    graterThan5 = 0
    graterThan10 = 0
    for key, value in badcase.items():
        if value >= 5:
            graterThan5 += 1
        if value >= 10:
            graterThan10 += 1

    return (test_accuracy, graterThan5, graterThan10)


def tramsform(num):
    """ 数字 转换成 字符

    :param num:
    :return:
    """
    if num < 10:
        return str(num)
    else:
        if num >= 24:
            num += 1
        if num >= 31:
            num += 1
        return chr(ord('A') + num - 10)


def conv_pool_operation(img, conv_W, conv_b):
    """CNN 卷积 和 池化操作
        一张图片
    :param img: array-3D
        一张图片，3D，(num_of_channels,img_height,img_width)
    :param conv_W: array-4D
        10*3*3
    :param conv_b:
    :return: array-3D
    """
    filter_row, filter_col = conv_W.shape[2:]
    img_row, img_col = img.shape[1:]
    # convolution
    # 3D
    conv_result = np.zeros((conv_W.shape[0], img_row - filter_row + 1, img_col - filter_col + 1))
    # quit()
    for filter_index in range(conv_W.shape[0]):
        for x in range(0, img_row - filter_row + 1):
            for y in range(0, img_col - filter_col + 1):
                i = img[0, x:x + filter_row, y:y + filter_col].flatten()
                j = conv_W[filter_index, 0].flatten()[-1::-1]
                conv_result[filter_index,
                            x,
                            y] = tanh_approximate_function(np.dot(i, j) + conv_b[filter_index])
                # print(np.dot(i, j) + conv_b[filter_index])

    # conv_result = tanh_approximate_function(conv_result)
    pool_row, pool_col = 2, 2
    pool_result = np.zeros(
        (conv_result.shape[0], conv_result.shape[1] / pool_row, conv_result.shape[2] / pool_col))
    # max-pooling
    for channel_index in range(pool_result.shape[0]):
        for x in range(pool_result.shape[1]):
            for y in range(pool_result.shape[2]):
                pool_result[
                    channel_index,
                    x,
                    y
                ] = np.max(
                    conv_result[
                    channel_index,
                    x * pool_row: (x + 1) * pool_row,
                    y * pool_col: (y + 1) * pool_col
                    ])
    return pool_result


def hidden_operation(feature_vector, W, b, activion='tanh'):
    """ 隐含层操作

    :param activion: str
        激活函数
    :param feature_vector: array-like
        一张 图片的 feature vector，1D
    :param W: array-2D
        权重
    :param b: array-1D
        偏差
    :return:
    """

    temp = np.dot(feature_vector, W) + b
    if activion == 'tanh':
        return np.asarray([tanh_approximate_function(item) for item in temp])
    elif activion == 'none':
        return temp
    else:
        raise NotImplementedError


def tanh_approximate_function(x):
    """
         * tanh x=sinh x / cosh x
         * 其中sinh x=(e^(x)-e^(-x))/2 ，cosh x=(e^x+e^(-x))/2
         * 所以tanhx = (e^(x)-e^(-x)) /(e^x+e^(-x))
    """
    #     return (int)(tanh(x));

    if x > 10:
        return 1
    elif x < -10:
        return -1
    else:
        return (int)((np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x)))


def cnn_batch_predict(X_val, weights):
    '''CNN批量预测

    :param X_val:
    :param weights:
    :return:
    '''
    weights = [(item * 1e6).astype(dtype=int) for item in weights]
    # result = cnn_predict(X_val[0],weights)
    result = []
    for index, img in enumerate(X_val):
        if (index + 1) % 1000 == 0:
            print('%d' % (index + 1))
        result.append(cnn_predict(img, weights))
    return np.asarray(result)


def cnn_predict(img, weights):
    '''单张图片的预测

    :param img:
    :param weights:
    :return:
    '''
    # 3D
    imgs_conv_result = conv_pool_operation(img, weights[0], weights[1])
    # print('conv over..')
    flatten_result = imgs_conv_result.flatten()
    # print('flatten over..')
    # print(flatten_result.shape)
    hidden1_output = hidden_operation(flatten_result, weights[2], weights[3], activion='tanh')

    # print('hidden1 over..')
    hidden2_output = hidden_operation(hidden1_output, weights[4], weights[5], activion='none')
    # print('hidden2 over..')
    # print(hidden2_output.shape)
    result = np.argmax(hidden2_output)
    return result



def cnn_batch_predict_spark(X_val_rdd,y_val, weights):
    weights = [(item * 1e6).astype(dtype=int) for item in weights]
#     result_rdd = X_val_rdd.map(lambda x:cnn_predict(x,weights))
    
    result_rdd = X_val_rdd.map(lambda x:conv_pool_operation(x, weights[0], weights[1]))
    result_rdd = result_rdd.map(lambda x: x.flatten())
    result_rdd = result_rdd.map(lambda x:hidden_operation(x,weights[2], weights[3],activion='tanh'))
    result_rdd = result_rdd.map(lambda x:hidden_operation(x,weights[4], weights[5],activion='none'))
    result_rdd = result_rdd.map(np.argmax)
    
#     result_rdd.cache()
    result = result_rdd.collect()
    accu = np.mean(np.asarray(result) == y_val) 
#     print(accu)
    return count_result(result,y_val)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11 µs


In [4]:
%%time
# region 读取数据集：验证数据(64369个)、测试数据(64381个)、其他应用数据集(243391个)
(X_val, y_val), (X_test, y_test), (X_other, y_other) = load_valdata(version='1122')

print(X_test.shape)
print(X_other.shape)

(64381, 1, 15, 15)
(243391, 1, 15, 15)
CPU times: user 4 s, sys: 300 ms, total: 4.3 s
Wall time: 4.41 s


### 将图片数据序列化

In [5]:
%%time
X_val_rdd = sc.parallelize(X_test)
y_val = y_test

CPU times: user 12 ms, sys: 16 ms, total: 28 ms
Wall time: 366 ms


In [6]:
%%time
X_val_rdd.cache

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.06 µs


<bound method RDD.cache of ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:475>

In [7]:
%%time
# X_val_rdd.count()
# len(y_val)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.06 µs


In [ ]:
%%time
with open(os.path.join(model_root_path, 'model_weight.pkl'), 'r') as fin:
    for index in range(1, len(model_file_list_path) + 1):
        # 从 模型1 开始，依次往后
        # 找到对应模型文件

        weights = pickle.load(fin)

        # print('OK')
        print(index,cnn_batch_predict_spark(X_val_rdd,y_val, weights))
#         print(test_accuracy, graterThan5, graterThan10)
        

In [15]:
%%time 

for i in range(10):
    %time pass
pass

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.81 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.96 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.81 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.05 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.81 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.05 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.01 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.96 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.01 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.96 µs
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 4.54 ms
